In [ ]:
corr_df = catalog.load('reflex_m3_wide')
m3_stock = catalog.load('m3_stock_parquet')
rf_stock = catalog.load('reflex_stock_parquet')

[12/09/25 15:15:44] INFO     Loading data from reflex_m3_wide (ParquetDataset)...              ]8;id=47076;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=633963;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

                    INFO     Loading data from m3_stock_parquet (ParquetDataset)...            ]8;id=779291;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=779657;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

                    INFO     Loading data from reflex_stock_parquet (ParquetDataset)...        ]8;id=418694;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=640977;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

In [ ]:
print("stock Reflex = ",rf_stock.Stock_en_VL.sum(), " - verif regul = ", corr_df.qty_reflex.sum())
print("stock M3 = ",m3_stock.Quantite.sum(), " - verif regul = ", corr_df.stock_total_m3.sum())

stock Reflex =  4024026.0  - verif regul =  4024026.0
stock M3 =  4019237.0  - verif regul =  3952792.0


In [3]:
rf_stock.where(rf_stock.Lot_1 == "").dropna()

,SKU,Qualite_Origine,Emplacement,Stock_en_VL,Lot_1
4,SCDIVERS003X,STD,STOCK,17.0,
42,DIPREPAETIQ0002,STD,STOCK,5674.0,
49,CAT000000000036,STD,STOCK,108.0,
87,SCPREPAETIQ00AH,STD,STOCK,187.0,
99,DIV000000000465,STD,STOCK,206.0,
...,...,...,...,...,...
21312,BDDISPLAY002G,STD,STOCK,4.0,
21321,PLMBOI000007936,STD,STOCK,4703.0,
21483,SCPREPAETIQ0055,STD,STOCK,4360.0,
21547,DIS000000000024,DES,DES,1.0,


In [6]:
m3_stock[m3_stock.SKU == "85XAO96JQZ00TUX"]

,SKU,Depot,Emplacement,Lot,Quantite,WMS
3682,85XAO96JQZ00TUX,150,STOCK,,6143.0,N/A
11043,85XAO96JQZ00TUX,100,STOCK,,6104.0,N/A
19254,85XAO96JQZ00TUX,100,NDISP,,1.0,N/A


In [13]:
corr_df

,sku,lot,qualite,reflex_emplacement,category,qty_reflex,stock_100,stock_150,stock_400,stock_total_m3,ecart_rfx_m3
0,BD60XAK840005,WF24-0057,STD,STOCK,STOCK,365.0,365.0,0.0,0.0,365.0,0.0
1,60PAN77JQZ4244,1000002553,STD,STOCK,STOCK,7.0,11.0,0.0,0.0,11.0,-4.0
2,SC60PAF320008,WF20-0243,DES,DES,DES,1.0,0.0,0.0,0.0,0.0,1.0
3,SC70A921200W2,WF24-0133A,RET,NDISP,NDISP,2.0,2.0,0.0,0.0,2.0,0.0
4,SC56LAG98ECH000F,SWF24-0028,STD,STOCK,STOCK,16.0,0.0,0.0,16.0,16.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
21767,SCPREPAETIQ00AQ,None,STD,STOCK,STOCK,200.0,200.0,0.0,0.0,200.0,0.0
21768,SCPREPAETIQ00AR,None,STD,STOCK,STOCK,200.0,200.0,0.0,0.0,200.0,0.0
21769,WODIVERS000N,None,STD,STOCK,STOCK,2378.0,2378.0,0.0,0.0,2378.0,0.0
21770,WOPLAQUETTE0004,None,STD,STOCK,STOCK,174.0,174.0,0.0,0.0,174.0,0.0


In [12]:
df = corr_df[corr_df.ecart_rfx_m3 != 0]

In [15]:
df.sort_values('ecart_rfx_m3')

,sku,lot,qualite,reflex_emplacement,category,qty_reflex,stock_100,stock_150,stock_400,stock_total_m3,ecart_rfx_m3
21521,85XAO96JQZ00TUX,None,STD,STOCK,STOCK,6054.0,6104.0,6143.0,0.0,12247.0,-6193.0
21525,86XAP26JIQ00TUX,None,STD,STOCK,STOCK,4065.0,4063.0,4083.0,0.0,8146.0,-4081.0
21523,85XAP80KEA00TUX,None,STD,STOCK,STOCK,3092.0,3639.0,3389.0,0.0,7028.0,-3936.0
21524,86XAP23JYH00TUX,None,STD,STOCK,STOCK,1601.0,1612.0,1603.0,0.0,3215.0,-1614.0
14693,56LAG98JTD0XXL,1000002965,STD,STOCK,STOCK,1949.0,1949.0,1152.0,0.0,3101.0,-1152.0
...,...,...,...,...,...,...,...,...,...,...,...
20114,619212JMZ4648,1000002314,STD,STOCK,STOCK,903.0,0.0,0.0,0.0,0.0,903.0
13740,64230NOZ0042,1000002390,BLO,REJET,REJET,945.0,0.0,0.0,0.0,0.0,945.0
2927,64230NOZ0044,1000002390,BLO,REJET,REJET,945.0,0.0,0.0,0.0,0.0,945.0
13776,85XAP79GCA00TU,1000003001,STD,STOCK,STOCK,5018.0,0.0,0.0,0.0,0.0,5018.0


In [24]:
m3_stock[m3_stock.SKU == "619212JMZ4648"]

,SKU,Depot,Emplacement,Lot,Quantite,WMS
19190,619212JMZ4648,200,DES,1000002314,2.0,N/A
23854,619212JMZ4648,100,STOCK,1000002013,415.0,N/A
24366,619212JMZ4648,150,STOCK,1000002013,408.0,N/A
26295,619212JMZ4648,100,STOCK,1000002344,658.0,N/A
